Load the pre-trained speaker diarization pipeline locally on our device:

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
api_key = os.getenv("HUGGING_FACE_AUTH")

from pyannote.audio import Pipeline
diarization_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization@2.1", use_auth_token=api_key
)

INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
c:\Users\Eric\AppData\Local\Programs\Python\Python312\Lib\inspect.py:1000: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\Eric\.cache\torch\pyannote\models--pyannote--segmentation\snapshots\c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b\pytorch_model.bin`
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching f

Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cpu. Bad things might happen unless you revert torch to 1.x.


hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

c:\Users\Eric\AppData\Local\Programs\Python\Python312\Lib\site-packages\speechbrain\utils\fetching.py:151: UserWarning: Using SYMLINK strategy on Windows for fetching potentially requires elevated privileges and is not recommended. See `LocalStrategy` documentation.
  warnings.warn(
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
c:\Users\Eric\AppData\Local\Programs\Python\Python312\Lib\site-packages\speechbrain\utils\autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)
c:\Users\Eric\AppData\Local\Programs\Python\Python312\Lib\site-packages\speechbrain\utils\parameter_transfer.py:234: UserWarning: Requested Pretrainer collection using symlinks on Windows. This might not work; see `LocalStrategy` documentation. Consider unsetting `colle

Load the mp3 that we have created with 3 speakers in 4 tracks all concatenated together.

In [ ]:
import soundfile as sf

filename = "combined_smaller"
combined_sample, combined_sample_rate = sf.read(f'combined/{filename}.wav')

print(combined_sample_rate)
print(combined_sample)


We can listen to the audio to see what it sounds like:

In [ ]:
from IPython.display import Audio

Audio(combined_sample, rate=combined_sample_rate)

Note that pyannote.audio expects the audio input to be a PyTorch tensor of shape (channels, seq_len),
so we need to perform this conversion prior to running the model:

In [ ]:
import torch

input_tensor = torch.from_numpy(combined_sample[None, :]).float()
outputs = diarization_pipeline(
    {"waveform": input_tensor, "sample_rate": combined_sample_rate}
)


# annotation_dict_list = list();
# for segment in outputs.itersegments():
#         annotation_dict_list.append({
#                 "segment": segment,
#                 "track": outputs.get_tracks(segment),
#                 "label": outputs.get_labels(segment)
#         })

# outputs.for_json()["content"]       
# for annotation in annotation_dict_list:
#         print(annotation)


diarization_output = []
for segment, track, label in outputs.itertracks(yield_label=True):
    diarization_output.append({'segment': {'start': segment.start, 'end': segment.end},
                        'track': track,
                        'label': label})
    
for segment in diarization_output:
    print(segment)

Use the Whisper model for our speech transcription system.

In [ ]:
from transformers import pipeline

asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-base",
)

Get the transcription for our sample audio, returning the segment level timestamps as well so that we know the start / end times for each segment.

Whiper does not work well with audio files longer than 30 seconds.

In [ ]:
asr_output = asr_pipeline(
    combined_sample.copy(),
    generate_kwargs={"max_new_tokens": 256},
    return_timestamps=True,
)

print(asr_output)

Find the closest alignment between diarization and transcription timestamps by minimising the absolute distance between both.

In [ ]:
from speechbox_trycatch_upto_idx import ASRDiarizationPipeline

pipeline = ASRDiarizationPipeline(
    asr_pipeline=asr_pipeline, diarization_pipeline=diarization_pipeline
)

Pass the audio file to the composite pipeline and see what we get out:

In [ ]:
# ASRDiarizationPipeline calls both models again. The boxing part has been stripped out of ASRDiarizationPipeline into box.ipynb.
# If hooking this up to the application will want to either use the box.ipynb method or use the below without calling the above.
final_output = pipeline(combined_sample.copy())

print(final_output)

In [ ]:
import json

all_output = {
    "speakers": final_output,
    "asr": asr_output["chunks"],
    "diarization": diarization_output
}

with open(f"output/{filename}.json", "w") as f:
    json.dump(all_output, f, indent=4)